<a href="https://colab.research.google.com/github/Kruthika0510/-Plagiarism-Checker/blob/main/Plagiarism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers datasets sklearn


  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
pip install --upgrade pip setuptools wheel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [1]:
pip install scikit-learn


In [15]:
pip install nltk


In [16]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from datasets import load_dataset
import pandas as pd

# Ensure nltk is ready
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!pip install datasets

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [10]:
from datasets import load_dataset

# Load PAWS dataset using a valid configuration name
dataset = load_dataset("paws", "labeled_final")  # Or 'labeled_swap' or 'unlabeled_final'

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [12]:
from datasets import load_dataset

# Load PAWS dataset
dataset = load_dataset("paws", "labeled_final")

# Access the train split using the 'train' key
train_dataset_df = dataset['train'].to_pandas()
train_dataset_df.to_csv("paws_train.csv", index=False)

# Access the validation and test splits similarly
val_dataset_df = dataset['validation'].to_pandas()
val_dataset_df.to_csv("paws_val.csv", index=False)

test_dataset_df = dataset['test'].to_pandas()
test_dataset_df.to_csv("paws_test.csv", index=False)

In [13]:
# Inspect the headers of the train dataset
print(train_dataset_df.columns)


Index(['id', 'sentence1', 'sentence2', 'label'], dtype='object')


In [14]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text pairs
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [1]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")

# Load stopwords
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Example usage
sample_text = "This is an example sentence, with some numbers 123!"
cleaned_text = clean_text(sample_text)
print(cleaned_text)  # Output: "example sentence numbers"


example sentence numbers


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from nltk.tokenize import word_tokenize

# Before using word_tokenize, download the 'punkt_tab' data:
import nltk
nltk.download('punkt_tab')

def tokenize_text(text):
    return word_tokenize(text)

# Example
tokens = tokenize_text(cleaned_text)
print(tokens)  # Output: ['example', 'sentence', 'numbers']

['example', 'sentence', 'numbers']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from nltk.tokenize import word_tokenize
nltk.download("punkt")

def tokenize_text(text):
    return word_tokenize(text)

# Example
tokens = tokenize_text(cleaned_text)
print(tokens)  # Output: ['example', 'sentence', 'numbers']

['example', 'sentence', 'numbers']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def calculate_bleu_score(reference, hypothesis):
    # Tokenize the sentences (you could use word-level or sentence-level tokenization)
    reference_tokens = nltk.word_tokenize(reference.lower())
    hypothesis_tokens = nltk.word_tokenize(hypothesis.lower())

    # Calculate BLEU score
    return sentence_bleu([reference_tokens], hypothesis_tokens)


In [10]:
## Load the PAWS dataset using a valid configuration name
from datasets import load_dataset
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Make sure to import or define calculate_bleu_score here
def calculate_bleu_score(reference, hypothesis):
    # Tokenize the sentences (you could use word-level or sentence-level tokenization)
    reference_tokens = nltk.word_tokenize(reference.lower())
    hypothesis_tokens = nltk.word_tokenize(hypothesis.lower())

    # Calculate BLEU score
    return sentence_bleu([reference_tokens], hypothesis_tokens)
dataset= load_dataset("paws", "labeled_final")  # Use 'labeled_final', 'labeled_swap', or 'unlabeled_final'

# Example: Calculate BLEU score for a few pairs in the training dataset
train_dataset = dataset['train']

# ... (rest of your code) .

bleu_scores = []

for example in train_dataset:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']

    # Calculate BLEU score
    bleu_score = calculate_bleu_score(sentence1, sentence2)
    bleu_scores.append(bleu_score)

# Convert the results into a pandas DataFrame
bleu_df = pd.DataFrame({'sentence1': [example['sentence1'] for example in train_dataset],
                        'sentence2': [example['sentence2'] for example in train_dataset],
                        'bleu_score': bleu_scores})

# Display the first few rows of the BLEU scores
print(bleu_df.head())
# ... (rest of your code) .

bleu_scores = []

for example in train_dataset:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']

    # Calculate BLEU score
    bleu_score = calculate_bleu_score(sentence1, sentence2)
    bleu_scores.append(bleu_score)

# Convert the results into a pandas DataFrame
bleu_df = pd.DataFrame({'sentence1': [example['sentence1'] for example in train_dataset],
                        'sentence2': [example['sentence2'] for example in train_dataset],
                        'bleu_score': bleu_scores})

# Display the first few rows of the BLEU scores
print(bleu_df.head())


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

                                           sentence1  \
0  In Paris , in October 1560 , he secretly met t...   
1  The NBA season of 1975 -- 76 was the 30th seas...   
2  There are also specific discussions , public p...   
3  When comparable rates of flow can be maintaine...   
4  It is the seat of Zerendi District in Akmola R...   

                                           sentence2  bleu_score  
0  In October 1560 , he secretly met with the Eng...    0.621813  
1  The 1975 -- 76 season of the National Basketba...    0.604275  
2  There are also public discussions , profile sp...    0.343893  
3  The results are high when comparable flow rate...    0.310833  
4  It is the seat of the district of Zerendi in A...    0.526640  


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

                                           sentence1  \
0  In Paris , in October 1560 , he secretly met t...   
1  The NBA season of 1975 -- 76 was the 30th seas...   
2  There are also specific discussions , public p...   
3  When comparable rates of flow can be maintaine...   
4  It is the seat of Zerendi District in Akmola R...   

                                           sentence2  bleu_score  
0  In October 1560 , he secretly met with the Eng...    0.621813  
1  The 1975 -- 76 season of the National Basketba...    0.604275  
2  There are also public discussions , profile sp...    0.343893  
3  The results are high when comparable flow rate...    0.310833  
4  It is the seat of the district of Zerendi in A...    0.526640  


In [8]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from datasets import load_dataset
import pandas as pd

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')

# Ensure nltk is ready
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
from transformers import TrainingArguments

# Disable logging to wandb
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    run_name="paws_run_1",           # custom run name (optional)
    evaluation_strategy="epoch",     # evaluate every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    report_to="none"                 # Disable wandb logging
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import TrainingArguments # Import TrainingArguments

# Load the PAWS dataset using a valid configuration name
dataset = load_dataset("paws", "labeled_final")

# Load a pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Load a pre-trained tokenizer  (Make sure tokenizer is defined before using it)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text pairs
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True) # Tokenize the dataset

# Define training_args here before using it in the Trainer
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    run_name="paws_run_1",           # custom run name (optional)
    eval_strategy="epoch",           # Use eval_strategy instead of evaluation_strategy
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=2,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    report_to="none"                 # Disable wandb logging
)

# Create the Trainer object
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=tokenized_datasets["train"],      # Pass the tokenized training dataset
    eval_dataset=tokenized_datasets["validation"],  # Pass the tokenized validation dataset
    tokenizer=tokenizer,                # Pass the tokenizer object instead of a string
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-2-a577f2431ef9>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
